# Task BONUS

**Import packages**

In [1]:
import sys
sys.path.append('../')
import tensorflow as tf
from tensorflow.keras import layers, Input, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from utils.DataLoader import loadData
from utils import Plots
import numpy as np

**Define N-Layer Residual network**

In [2]:
def resnet(input_shape, num_layers):
    inputs = Input(shape=input_shape)
    x = layers.Conv2D(64, (7, 7), padding="same", activation="relu")(inputs)
    x = layers.MaxPooling2D((3, 3))(x)

    for _ in range(num_layers):
        # residual block
        residual = x
        x = layers.Conv2D(64, (3, 3), padding="same", activation="relu")(x)
        x = layers.Conv2D(64, (3, 3), padding="same")(x)
        x = layers.add([x, residual])
        x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(9, activation="softmax")(x)

    return Model(inputs, x)

**LoadData**

In [3]:
img_w, img_h, img_ch = 128, 128, 1
rel_path = 'Lab1/X_ray/'

# load data
x_train, x_test, y_train, y_test = loadData(img_w, img_h, rel_path)

Reading: 0/5780 of train images
Reading: 100/5780 of train images
Reading: 200/5780 of train images
Reading: 300/5780 of train images
Reading: 400/5780 of train images
Reading: 500/5780 of train images
Reading: 600/5780 of train images
Reading: 700/5780 of train images
Reading: 800/5780 of train images
Reading: 900/5780 of train images
Reading: 1000/5780 of train images
Reading: 1100/5780 of train images
Reading: 1200/5780 of train images
Reading: 1300/5780 of train images
Reading: 1400/5780 of train images
Reading: 1500/5780 of train images
Reading: 1600/5780 of train images
Reading: 1700/5780 of train images
Reading: 1800/5780 of train images
Reading: 1900/5780 of train images
Reading: 2000/5780 of train images
Reading: 2100/5780 of train images
Reading: 2200/5780 of train images
Reading: 2300/5780 of train images
Reading: 2400/5780 of train images
Reading: 2500/5780 of train images
Reading: 2600/5780 of train images
Reading: 2700/5780 of train images
Reading: 2800/5780 of train imag

**Balance training set and create folds**

In [4]:
class_numerosity = np.sum(y_train, axis=0)
num_classes = len(class_numerosity)
smallest = np.min(class_numerosity)

# for each class, choose at random a number of samples equal to the least represented class
sampled_indices1, sampled_indices2, sampled_indices3 = [], [], []
for i in range(num_classes):
    class_indices = np.array(np.where(y_train[:, i] == 1)).flatten()
    sampled_class_indices = np.random.choice(class_indices, size=smallest, replace=True)
    # create folds
    indices1, indices2, indices3 = np.array_split(sampled_class_indices, 3)
    sampled_indices1.append(indices1)
    sampled_indices2.append(indices2)
    sampled_indices3.append(indices3)

# stack the indices together
fold1 = np.hstack(sampled_indices1)
fold2 = np.hstack(sampled_indices2)
fold3 = np.hstack(sampled_indices3)

# shuffle the indices of the balanced set
np.random.shuffle(fold1), np.random.shuffle(fold2), np.random.shuffle(fold3)

x_fold1 = x_train[fold1, :, :, :]
y_fold1 = y_train[fold1, :]
x_fold2 = x_train[fold2, :, :, :]
y_fold2 = y_train[fold2, :]
x_fold3 = x_train[fold3, :, :, :]
y_fold3 = y_train[fold3, :]

**Define the model**

In [37]:
# depth of model
num_layers = 10
# learning rate
learning_rate = 0.001
# batch size
batch_size = 8

# define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# define learning rate schedule
def lr_schedule(epoch):
    if epoch < 20:
        return 0.001
    else:
        return 0.0001

lr_scheduler = LearningRateScheduler(lr_schedule)

# compile the model
model = resnet((img_w, img_h, img_ch), num_layers)
model.compile(
    optimizer=Adam(learning_rate = learning_rate), loss="categorical_crossentropy", metrics=["accuracy"]
)
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_126 (Conv2D)            (None, 128, 128, 64  3200        ['input_7[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d_6 (MaxPooling2D)  (None, 42, 42, 64)  0           ['conv2d_126[0][0]']             
                                                                                            

**Use 3-fold cross-validation approach to fit**

In [38]:
fold_accuracy = []
for i in range(1,4,1):
    if i == 1:
        x_train = np.vstack([x_fold2, x_fold3])
        y_train = np.vstack([y_fold2, y_fold3])
        x_valid = x_fold1
        y_valid = y_fold1
        
    elif i == 2:
        x_train = np.vstack([x_fold1, x_fold3])
        y_train = np.vstack([y_fold1, y_fold3])
        x_valid = x_fold2
        y_valid = y_fold2
        
    else:
        x_train = np.vstack([x_fold1, x_fold2])
        y_train = np.vstack([y_fold1, y_fold2])
        x_valid = x_fold3
        y_valid = y_fold3
    # train the model
    print(f"Using fold {i} as validation set...")
    model_fit = model.fit(x_train, y_train, batch_size=batch_size, epochs=100, verbose=0, validation_data=[x_valid, y_valid], callbacks=[early_stopping, lr_scheduler])    
    
    fold_accuracy.append(np.max(model_fit.history["val_accuracy"]))

Using fold 1 as validation set...
Using fold 2 as validation set...
Using fold 3 as validation set...


In [39]:
fold_mean = np.mean(fold_accuracy)
fold_std = np.std(fold_accuracy)
print(f"Accuracy: {fold_mean:.3f}+-{fold_std:.3f}")

Accuracy: 0.977+-0.008
